In [ ]:
from sdpa_xla import XLASDPAWrapper,SDPA_ATTENTION_AVAILABLE
from transformers.models.qwen2.modeling_qwen2 import apply_rotary_pos_emb

model = AutoModelForCausalLM.from_pretrained(MODEL,torch_dtype=torch.bfloat16,trust_remote_code=True)

for layer in model.model.layers:
    xs.apply_backward_optimization_barrier(layer)
    
with torch.no_grad():
    if SDPA_ATTENTION_AVAILABLE:
        old_atten_config=[]
        
        mesh = xs.get_global_mesh() #Currently the mesh is (4,1)
        partition_spec = (0, None,None,None) ## Always try to shard along Batch Dimension 
        
        """
        Currently not sharding activation ; not showing any improvements ; if improvements are seen then we can shard the activation as well
        """
        
        # Replace attention mechanism with XLA Flash Attention
        for layer in model.model.layers:
            original_attention = layer.self_attn
            layer.self_attn = XLASDPAWrapper(original_attention, mesh, partition_spec,rotary_func=apply_rotary_pos_emb)
            old_atten_config.append(original_attention)
        print("Applied SDPA Attention for TPU ; Saved Old Attention config for replacement after training")


In [ ]:
#train...

In [ ]:
# Unwrap the XLA SDPA Attention
model = model.cpu()
if SDPA_ATTENTION_AVAILABLE :
    try:
        for index,attn in enumerate(old_atten_config):
            model.model.layers[index].self_attn=attn
        print("Replaced XLASDPAWrapper With Original Attention Implementation")
    except:
        print("Unable to Replace XLASDPAWrapper With Original Attention Implementation")
    model = model.cpu()